In [ ]:
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import numpy as np
from pathlib import Path
from bmcs_cross_section.api import ACI440

home_dir = Path().home()
ml_data = home_dir / 'ml_data'
if not ml_data.exists():
    ml_data.mkdir()

In [ ]:
import numpy as np
import plotly.graph_objs as go

rho_b = ACI440.get_rho_balanced(f_c=50, f_fu=900, E_f=70000)*100
# psi_c = ACI440.get_psi_c(rho, A_f, f_fu=900, E_f=70000, f_c=50, b, d)
# psi_f = ACI440.get_psi_f(rho, f_fu=900, E_f=70000, f_c=50)

# Load data 
Design_space = np.load(ml_data / 'designspace_meshgrid_data.npz')
Mn_aci = np.load(ml_data / 'ACI440_Mn.npy')
rho_mesh, b_mesh, d_mesh = Design_space['rho_mesh'], Design_space['b_mesh'], Design_space['d_mesh']

mean_Mn_aci = np.mean(Mn_aci)
max_Mn_aci = np.max(Mn_aci)
min_Mn_aci =  np.min(Mn_aci)


target_Mn = np.linspace(min_Mn_aci, max_Mn_aci, 7)
print(f"Target Mn values: {target_Mn}")
rho_target = 2*rho_b  # Replace rho_b with your actual rho_target if different

# Define the tolerance
rtol = 0.01
data = []
for i, target in enumerate(target_Mn):
    indices = np.where(np.abs(Mn_aci - target) / target < rtol)
    admissible_sets = np.column_stack([rho_mesh[indices], b_mesh[indices], d_mesh[indices]])
    print(f"Number of admissible sets for target_Mn = {target}: {len(admissible_sets)}")

    if len(admissible_sets) > 0:
        p_values = admissible_sets[:, 0]
        b_values = admissible_sets[:, 1]
        d_values = admissible_sets[:, 2]

        # mask = p_values > rho_b
        # p_values = p_values[mask]
        # b_values = b_values[mask]
        # d_values = d_values[mask]
        
        scatter = go.Scatter3d(
            x=p_values,
            y=b_values,
            z=d_values,
            mode='markers',
            marker=dict(
                size=1,
                color='blue',
                opacity=0.4
            ),
        )

        data.append(scatter)

        # Filter points with rho within 1% of rho_target
        mask_rho_target = np.abs(p_values - rho_target) / rho_target < rtol
        p_values_rho_target = p_values[mask_rho_target]
        b_values_rho_target = b_values[mask_rho_target]
        d_values_rho_target = d_values[mask_rho_target]
        scatter_rho_target = go.Scatter3d(
            x=p_values_rho_target,
            y=b_values_rho_target,
            z=d_values_rho_target,
            mode='markers',
            marker=dict(
                size=2,
                opacity=0.8
            ),
        )

        data.append(scatter_rho_target)

# Create a mesh grid for b and d values
b_range = np.linspace(np.min(b_mesh), np.max(b_mesh), 50)
d_range = np.linspace(np.min(d_mesh), np.max(d_mesh), 50)
b_grid, d_grid = np.meshgrid(b_range, d_range)

# Create a plane of rho_b
rho_b_plane = rho_b * np.ones_like(b_grid)

# Add surface plot of rho_b
surface = go.Surface(
    x=rho_b_plane,
    y=b_grid,
    z=d_grid,
    colorscale='Viridis',
    opacity=0.5,
    name='Balanced Reinforcement Ratio'
)

data.append(surface)

layout = go.Layout(
    title='Admissible Sets for Mn_ACI440(rho, b, d) = target_Mn',
    scene=dict(
        xaxis=dict(title='rho (%)'),
        yaxis=dict(title='Width (b) (mm)'),
        zaxis=dict(title='Depth (d) (mm)')
    )
)

fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
import numpy as np
import plotly.graph_objs as go


rho_b = ACI440.get_rho_balanced(f_c=50, f_fu=900, E_f=70000)*100
# psi_c = ACI440.get_psi_c(rho, A_f, f_fu=900, E_f=70000, f_c=50, b, d)
# psi_f = ACI440.get_psi_f(rho, f_fu=900, E_f=70000, f_c=50)

# Load design space data
Design_space = np.load(ml_data / 'designspace_meshgrid_data.npz')
Mn_aci = np.load(ml_data / 'ACI440_Mn.npy')
rho_mesh, b_mesh, d_mesh = Design_space['rho_mesh'], Design_space['b_mesh'], Design_space['d_mesh']



mean_Mn_aci = np.mean(Mn_aci)
target_Mn = np.array([1.5*mean_Mn_aci])

print(f"Target Mn values: {target_Mn}")

rtol = 0.01

data = []
colors = ['blue', 'green', 'red']
for i, target in enumerate(target_Mn):
    indices = np.where(np.abs(Mn_aci - target)/target < rtol)

    admissible_sets = np.column_stack([rho_mesh[indices], b_mesh[indices], d_mesh[indices]])
    print(f"Number of admissible sets for target_Mn = {target}: {len(admissible_sets)}")

    if len(admissible_sets) > 0:
        p_values = admissible_sets[:, 0]
        b_values = admissible_sets[:, 1]
        d_values = admissible_sets[:, 2]

        scatter = go.Scatter3d(
            x=p_values,
            y=b_values,
            z=d_values,
            mode='markers',
            marker=dict(
                size=1,
                color=colors[i],
                opacity=0.8
            ),
            name=f'Target Mu = {target:.2f}'
        )

        data.append(scatter)

# Create a mesh grid for b and d values
b_range = np.linspace(np.min(b_mesh), np.max(b_mesh), 50)
d_range = np.linspace(np.min(d_mesh), np.max(d_mesh), 50)
b_grid, d_grid = np.meshgrid(b_range, d_range)

# Create a plane of rho_b
rho_b_plane = rho_b * np.ones_like(b_grid)

# Add surface plot of rho_b
surface = go.Surface(
    x=rho_b_plane,
    y=b_grid,
    z=d_grid,
    colorscale='Viridis',
    opacity=0.5,
    name='Balanced Reinforcement Ratio'
)

data.append(surface)

layout = go.Layout(
    title='Admissible Sets for Mn_ACI440(rho, b, d) = target_Mn',
    scene=dict(
        xaxis=dict(title='rho (%)'),
        yaxis=dict(title='Width (b) (mm)'),
        zaxis=dict(title='Depth (d) (mm)')
    )
)

fig = go.Figure(data=data, layout=layout)
fig.show()


In [ ]:
from scipy.optimize import curve_fit
from scipy.interpolate import Rbf
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

Design_space = np.load(ml_data / 'designspace_meshgrid_data.npz')
Mn_aci = np.load(ml_data / 'ACI440_Mn.npy')
rho_mesh, b_mesh, d_mesh = Design_space['rho_mesh'], Design_space['b_mesh'], Design_space['d_mesh']

mean_Mn_aci = np.mean(Mn_aci)
# target_Mn = np.array([.2*mean_Mn_aci, mean_Mn_aci, 1.8*mean_Mn_aci])
target_Mn = np.array([mean_Mn_aci])

print(f"Target Mn values: {target_Mn}")

rtol= 0.01

indices = np.where(np.abs(Mn_aci - target_Mn) < rtol*target_Mn)
admissible_sets = np.column_stack([rho_mesh[indices], b_mesh[indices], d_mesh[indices]])
print(f"Number of admissible sets: {len(admissible_sets)}")

rho = admissible_sets[:, 0]
b = admissible_sets[:, 1]
d = admissible_sets[:, 2]

X = np.column_stack((b, d))
poly = PolynomialFeatures(degree=4)  
X_poly = poly.fit_transform(X)

model = LinearRegression()
model.fit(X_poly, rho)

# Generate a grid of points to evaluate the fitted surface
b_fit = np.linspace(min(b), max(b), 100)
d_fit = np.linspace(min(d), max(d), 100)
b_fit, d_fit = np.meshgrid(b_fit, d_fit)
X_fit = np.column_stack((b_fit.ravel(), d_fit.ravel()))
X_fit_poly = poly.transform(X_fit)
p_fit = model.predict(X_fit_poly).reshape(b_fit.shape)

fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=b,
    y=d,
    z=rho,
    mode='markers',
    marker=dict(size=1, color='red'),
    name='Data points'
))

fig.add_trace(go.Surface(
    x=b_fit,
    y=d_fit,
    z=p_fit,
    colorscale='Viridis',
    opacity=0.6,
    name='Fitted surface'
))

# Update layout for better visualization
fig.update_layout(
    scene=dict(
        xaxis_title='b',
        yaxis_title='d',
        zaxis_title='rho',
        xaxis=dict(range=[0, np.max(b_mesh)]),
        yaxis=dict(range=[0, np.max(d_mesh)]),
        zaxis=dict(range=[0, np.max(rho_mesh)]),

    ),
    title='3D Polynomial Surface Fit',
    showlegend=True
)

fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go

# Define the grid
u = np.linspace(0, np.pi/2, 100)  # Range for one-quarter of the ellipsoid
v = np.linspace(0, np.pi/2, 100)  # Range for one-quarter of the ellipsoid
u, v = np.meshgrid(u, v)

# Define the semi-principal axes
a, b, c = 2, 1, 3

# Parametric equations for the ellipsoid surface
x = a * np.cos(u) * np.sin(v)
y = b * np.sin(u) * np.sin(v)
z = c * np.cos(v)

# Create the 3D surface plot
fig = go.Figure(data=[go.Surface(x=x, y=y, z=z, colorscale='Viridis')])

# Update layout for better visualization
fig.update_layout(
    scene=dict(
        xaxis_title='X-axis',
        yaxis_title='Y-axis',
        zaxis_title='Z-axis',
        aspectratio=dict(x=1, y=1, z=1),
        camera_eye=dict(x=1.2, y=1.2, z=0.6)
    ),
    title='One-Quarter of the Ellipsoid Surface',
)

# Show the plot
fig.show()


In [ ]:
import numpy as np
from scipy.optimize import minimize
from bmcs_cross_section.api import ACI440

@np.vectorize
def Mn_ACI440(rho, b, d):
    A_f = rho * b * d / 100
    Mn = ACI440.get_M_n(A_f=A_f, f_fu=900, E_f=70000, f_c=50, b=b, d=d)
    return Mn

# Define the objective function for the optimizer
def objective_function(params, target_Mn):
    rho, b, d = params
    Mn = Mn_ACI440(rho, b, d)
    return abs(Mn - target_Mn)

# Initial guess for the parameters
initial_guess = [0.2, 500, 500]

# Define bounds for the parameters (optional)
bounds = [(0, 2), (10, 1000), (10, 1000)]  # Example: parameters must be non-negative

# Define the target Mn value you want to find the inverse for
target_Mn = 1440  # Example target value

# Minimize the objective function with bounds
result = minimize(objective_function, initial_guess, args=(target_Mn,), bounds=bounds)

if result.success:
    inverse_params = result.x
    print(f"Inverse parameters for target Mn = {target_Mn}: rho = {inverse_params[0]}, b = {inverse_params[1]}, d = {inverse_params[2]}")
    print(f"Function output: {Mn_ACI440(*inverse_params)}, expected {target_Mn}")
else:
    print("Optimization failed.")
